In [10]:
Zn = Zmod(n)
G0= Zn.unit_group()
G0.inject_variables()

Defining f0, f1


In [11]:
Zn(f0)

11

In [12]:
Zn(f1)

7

In [114]:
Qn.<z>=CyclotomicField(n)

In [115]:
G_0 = Qn.galois_group()

In [143]:
K = Qn.subfield(z + z^-1)

## Definizione di $n$ e sua fattorizzazione

In [179]:
n=3*5*7*11
fn = factor(n)
fn

5 * 7 * 11

## Power set $P_S$

In [180]:
s = len(fn)
S = [ i +1 for i in range(s)]
PS = Subsets(S).list()
PS.remove(Set(S))

In [181]:
PS

[{}, {1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}]

## Define the group $G_0$

In [182]:
Qn.<z>=CyclotomicField(n)
G_0 = Qn.galois_group()

In [210]:
G_0.an_element().as_hom()

Ring endomorphism of Cyclotomic Field of order 385 and degree 240
  Defn: z |--> z^123

In [ ]:
G_0.an_element().as_hom

## Group Ring
We assume that $G_0$ is the group to work with

In [117]:
A  = GroupAlgebra(G_0, ZZ)

In [ ]:
G_0.list()

## Definition of $\beta$

In [102]:
def beta0(i : int):
    return i

In [103]:
[ beta0(i) for i in S]

[1, 2]

In [108]:
def beta(I : set):
    r = 1
    for j in I:
        r = r * beta0(j)
    return r

# Calculation of the index
First we define the number field $K$ as the Maximal Real subfield of $\mathbb{Q}(\zeta_n)$

In [183]:
Fn.<z> = CyclotomicField(n)
K = Fn.subfield(z + z^-1)

We define $\epsilon_i$ knowing that is equal to $\phi(p_i^{e_i})$

In [184]:
def eps(i):
    return euler_phi(fn[i-1][0]**fn[i-1][1])

In [185]:
eps(2)

6

In [186]:
zz = z + z.conjugate()

In [187]:
K = NumberField(zz.minpoly(),'a')

In [188]:
def g(i):
    I = K.ideal(fn[i-1][0])
    return len(I.factor())

In [189]:
g(2)

1

For $f_i$ we use the equality $[K:\mathbb{Q}] = \epsilon_i g_i f_i $ since we know the other three elements.  
*Memo*: $[K:\mathbb{Q}] = \phi(n)/2 $

In [164]:
def f(i):
    return euler_phi(n)/(2*g(i)*eps(i))

In [168]:
f(2)

1

So we have that:

In [199]:
def i_b():
    i_b=1
    for i in S:
        i_b *= (eps(i)**(g(i)-1)) * (f(i)**(2*g(i) -1))
    return i_b

In [194]:
i = i_b(n)
i

1312200000

In [195]:
factor(i)

2^6 * 3^8 * 5^5

In [221]:
def i_b_compressed(n):
    fn = factor(n)
    s = len(fn)
    S = [ i +1 for i in range(s)]
    K.<z> = CyclotomicField(n)
    zz = z + z.conjugate()
    K = NumberField(zz.minpoly(),'a')
    ibb=1
    for j in S:
        eps = euler_phi(fn[j-1][0]**fn[j-1][1])
        g = len(K.ideal(fn[j-1][0]).factor())
        f = euler_phi(n)/(2*g*eps)
        ibb *= (eps**(g-1)) * (f**(2*g-1))
    return ibb

In [ ]:
factor(i_b_compressed(3*5*7*11))